In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.alert import Alert
from bs4 import BeautifulSoup
import requests, lxml
from re import search
from collections import Counter
import numpy as np
import threading 
import queue
import time

In [2]:
def get_driver():
    options = webdriver.FirefoxOptions()
    options.add_argument('-headless')
    options.add_argument('-no-sandbox')
    options.add_argument('-disable-dev-shm-usage')
    options.add_argument('--disable-application-cache')
    driver = webdriver.Firefox(options = options)
    return driver

In [4]:
def get_contractor(driver, taxid):
    url = 'https://dauthau.asia/businesslistings/search/?q=' + taxid + '&industry1=&province=0&businesstype=0&lvkd=0'
    driver.get(url)
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'lxml')
    link = soup.find('div', class_= 'listing-summary').a['href']
    driver.get('https://dauthau.asia' + link)
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')
    table = soup.find("table")
    df = pd.read_html(str(table), converters={'Mã số thuế': str, 'Điện thoại công ty': str})[0]
    df = df.transpose()
    df.columns = df.iloc[0]
    df = df.iloc[1:2]
    df = df.iloc[:,:17]
    driver.delete_all_cookies()
    time.sleep(2)
    return df


In [5]:
def get_contractor_list_driver(driver, tax_id_lst):
    df = get_contractor(driver, tax_id_lst[0])
    for i in tax_id_lst:
        if i != tax_id_lst[0]:
            try:
                df = pd.concat([df,get_contractor(driver, i)])
            except:
                continue
        else:
            pass
    return df

In [6]:
def get_contractor_list(tax_id_lst):
    driver = get_driver()
    df = get_contractor_list_driver(driver, tax_id_lst)
    driver.close()
    return df

In [6]:
# input_file = pd.read_excel("/home/dangpdh/Downloads/dauthau.xlsx", dtype = 'str')

In [7]:
import psycopg2

In [8]:
conn = psycopg2.connect(host="localhost",database="postgres", user="postgres", password="dang1199")

In [9]:
sql = """select distinct "Mã số thuế" from kq_dauthau_202208 kd 
        where "Mã số thuế" not in (select "TAX_ID" from contractor_info ci)"""

In [10]:
input_file = pd.read_sql(sql, conn)

/home/dangpdh/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [11]:
input_file = input_file.loc[input_file["Mã số thuế"].str.len() >= 10]

In [12]:
u_lst = list(set(input_file["Mã số thuế"].tolist()))

In [13]:
len(u_lst)

748

In [14]:
i = 0
while i < 748:
    print(i)
    start = time.time()
    print(start)
#     lst = u_lst[i:i+1000]
    lst = u_lst
    splits = np.array_split(lst, 2)
    FILE_QUEUE = queue.Queue()
    threads = []
    for n in range(2):
        t = threading.Thread(target = lambda q, arg1: q.put(get_contractor_list(arg1)), args = [FILE_QUEUE, splits[n]])
        t.start()
        threads.append(t)
    for t in threads:
        t.join()
    df = pd.DataFrame()
    while not FILE_QUEUE.empty():
        data = FILE_QUEUE.get()
        df = df.append(data, sort=False).reset_index(drop=True)
    df.to_csv("/home/dangpdh/Downloads/contractor_info_" + str(i) +".csv",)
    end = time.time()
    print(end - start)
    i += 1000

0
1662775232.5441444
80.25834107398987


/tmp/ipykernel_6200/3326941065.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, sort=False).reset_index(drop=True)
/tmp/ipykernel_6200/3326941065.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, sort=False).reset_index(drop=True)


In [34]:
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.core.utils import ChromeType
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium_stealth import stealth

In [47]:
options = webdriver.ChromeOptions()
# options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')
options.add_argument('--disable-application-cache')
options.add_argument("--remote-debugging-port=9222")
driver = webdriver.Chrome(ChromeDriverManager(chrome_type=ChromeType.CHROMIUM).install(),chrome_options=options)

[WDM] - ====== WebDriver manager ======


2022-09-15 01:46:41,502 INFO ====== WebDriver manager ======


[WDM] - Current chromium version is 105.0.5195


2022-09-15 01:46:42,164 INFO Current chromium version is 105.0.5195


[WDM] - Get LATEST chromedriver version for 105.0.5195 chromium


2022-09-15 01:46:42,165 INFO Get LATEST chromedriver version for 105.0.5195 chromium


[WDM] - Driver [/home/dangpdh/.wdm/drivers/chromedriver/linux64/105.0.5195.52/chromedriver] found in cache


2022-09-15 01:46:42,574 INFO Driver [/home/dangpdh/.wdm/drivers/chromedriver/linux64/105.0.5195.52/chromedriver] found in cache


/tmp/ipykernel_23537/4050824020.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager(chrome_type=ChromeType.CHROMIUM).install(),chrome_options=options)
/tmp/ipykernel_23537/4050824020.py:7: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager(chrome_type=ChromeType.CHROMIUM).install(),chrome_options=options)


In [48]:
stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
        )

In [49]:
df = get_contractor_list_driver(driver,u_lst)

In [50]:
df.describe()

,Mã số thuế,Lượt xem,Tên công ty,Tên quốc tế,Địa chỉ trụ sở,Điện thoại công ty,Fax công ty,Website công ty,Lĩnh vực kinh doanh,Loại doanh nghiệp,Ngày thành lập công ty,Vốn điều lệ,Số nhân viên,Trạng thái công ty,Ngày phê duyệt trên mạng đấu thầu quốc gia,Trạng thái nộp phí,Cập nhật lần cuối,Địa chỉ giao dịch
count,741,741,741,324,741,741,135,0,0,669,741,0,741,741,741,741,371,370
unique,741,275,741,324,740,740,134,0,0,5,680,0,85,1,531,17,367,369
top,1300997203,13,CÔNG TY TNHH MỘT THÀNH VIÊN THƯƠNG MẠI DỊCH VỤ...,NEW SOLUTION INVESTMENT CONSTRUCTION & COMMERC...,"Thôn 9, Xã Phùng Xá, Huyện Thạch Thất, Hà Nội",0553822730,02213,NaN,NaN,Công ty TNHH,18/03/2011,NaN,0,Đang hoạt động,04/08/2022,Đã nộp chi phí Hãy xác thực tài khoản trên Dau...,05:24 09/08/2022 - Cập nhật lần thứ 4. Vui lòn...,"Thôn 9, Xã Phùng Xá, Huyện Thạch Thất, Thành p..."
freq,1,14,1,1,2,2,2,NaN,NaN,398,4,NaN,246,741,6,472,2,2


In [51]:
df.to_csv("/home/dangpdh/Downloads/contractor_info_test.csv")

In [25]:
requests.get('https://dauthau.asia/businesslistings/search/?q=0941740708&industry1=&province=0&businesstype=0&lvkd=0')

<Response [403]>